In [13]:
import get_data
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from random import random

# AAPL Example

We first get all the relevant data. We care about adjusted close and all technical indicators. The variables `drop_days`, `drop_percent`, and `rec_days` are used for the label and selecting rows for the dataset. BUT FIRST pick start and end dates.

In [14]:
all_data = get_data.get_data('AAPL').sort_values('date').drop(["1. open", "2. high", "3. low", "4. close", "6. volume", "7. dividend amount", "8. split coefficient"], axis=1)
all_data['recovered'] = 0
all_data['drop'] = 0
all_data['in range'] = 0
all_data['test'] = [1 if random() < 0.7 else 0 for n in range(len(all_data))]

start_ind = 2380 # 3639 is 5 years, 2380 is 10 years
print(all_data.index[start_ind]) 

2009-12-30 00:00:00


In [15]:
final_set = []
rec_days = 30

drops = 0
recs = 0
for n in range(start_ind, len(all_data) - rec_days):
    date = all_data.index[n]
    all_data['in range'][n] = 1
    
    if all_data["5. adjusted close"][n-1] > all_data["5. adjusted close"][n]:
        all_data['drop'][n] = 1
        drops += 1
        if all_data["5. adjusted close"][n + rec_days] > all_data["5. adjusted close"][n]:
            recs += 1
            all_data['recovered'][n] = 1


train_data = all_data[all_data['in range'] == 1]
train_data = all_data[all_data['drop'] == 1]
train_data = train_data.drop(columns=['drop', 'in range', '5. adjusted close'])
len(train_data) == drops

C:\Users\yugan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yugan\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\yugan\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


True

## Training
We train the model. First, divide the training data into training and test data. Then preprocess.

In [16]:
test_data = train_data[train_data['test'] == 1].drop(columns=['test'])
train_data = train_data[train_data['test'] == 0].drop(columns=['test'])
print(len(test_data), len(train_data))

832 345


Train the model.

In [17]:
features = train_data.columns[:-1] # everything but 'recovered' is a feature
y = train_data['recovered']

rfc = RandomForestClassifier(n_jobs=-1, n_estimators = 1000)
rfc.fit(train_data[features], y)
predictions = rfc.predict(test_data[features])
len(predictions) == len(test_data)

True

In [18]:
eval = []
for n in range(len(predictions)):
    eval.append(int(predictions[n] == test_data['recovered'][n]))
sum(eval) / len(eval)

0.8040865384615384